# Collecting Threads 

In [ ]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="stackoverflow")

In [ ]:
keywords_to_search = [
    'mysql', 'postgresql', 'mariadb', 'mongodb', 'sqlite', 'redis', 'cassandra', 'cockroachdb', 'couchdb', 'neo4j', # FOSS DB
    'oracle database', 'microsoft sql server', 'ibm db2', # CSPS OS
    'linux', 'freebsd', 'openbsd', 'netbsd', 'android', # FOSS OS
    'microsoft windows', 'macos', 'ios', # CSPS OS
    'apache cloudstack', 'opennebula', 'proxmox ve', ' xen orchestra', # FOSS Virtualization
    'hyper-v', 'vmware workstation', 'vmware fusion'# CSPS Virtualization
] 

In [ ]:
# Get posts with comments and answers
def getQuery(keyword: str):
    return f"""
        WITH DockerPosts AS (
          SELECT id, title, body, tags, creation_date,
          FROM bigquery-public-data.stackoverflow.stackoverflow_posts
          WHERE 
            (LOWER(title) LIKE '%{keyword}%' OR
             LOWER(body) LIKE '%{keyword}%' OR
             LOWER(tags) LIKE '%{keyword}%')
             AND score > 0  
          LIMIT 500000
        )
        
        SELECT * FROM (
          SELECT 
            dp.id AS post_id,
            dp.title AS post_title,  -- This is STRING
            dp.creation_date AS post_creation_date,
            'Post' AS content_type,
            dp.body AS content,
            CAST(NULL AS STRING) AS parent_post_id  -- Ensure STRING type
          FROM DockerPosts dp
        
          UNION ALL
        
          SELECT 
            pa.id AS post_id,
            CAST(NULL AS STRING) AS post_title,  -- Convert to STRING
            pa.creation_date AS post_creation_date,
            'Answer' AS content_type,
            pa.body AS content,
            CAST(pa.parent_id AS STRING) AS parent_post_id  -- Ensure STRING type
          FROM DockerPosts dp
          JOIN bigquery-public-data.stackoverflow.posts_answers pa 
            ON dp.id = pa.parent_id 
          WHERE pa.score > 0 
        ) 
        
        UNION ALL
        
        SELECT * FROM (
          SELECT 
            c.id AS post_id,
            CAST(NULL AS STRING) AS post_title,  -- Convert to STRING
            c.creation_date AS post_creation_date,
            'Comment' AS content_type,
            c.text AS content,
            CAST(c.post_id AS STRING) AS parent_post_id  -- Ensure STRING type
          FROM DockerPosts dp
          JOIN bigquery-public-data.stackoverflow.comments c 
            ON dp.id = c.post_id
          WHERE c.score > 0  
        ) 
        
        ORDER BY post_creation_date;
        
        
        """

In [ ]:
for keyword in keywords_to_search:
    print(keyword)
    query = getQuery(keyword) 
    response = stackOverflow.query_to_pandas_safe(query,max_gb_scanned=71)
    response.to_csv(f'{keyword}_posts_with_comments_answers.csv', index=False)